## Part 2: GLM Example

本节的目的在与：了解如何通过广义线性模型(Generalized linear model)拟合正确率等二元决策变量。

重点在于：
- 了解使用 Pymc 进行数据分析的完整 workflow
- 了解因变量为正确率等二元决策变量(往往记录为0或1，1代表回答正确，0代表回答错误)的特征
- 了解广义线性模型(Generalized linear model)中的伯努利(Bernoulli)分布和链接函数(link function)

![Image Name](https://cdn.kesci.com/upload/image/rkvikqg9q6.png?imageView2/0/w/650/h/650)

### (1) 提出研究问题

Stroop 测试是一种注意力测试，可以用来检测反应抑制能力。

![](https://www.researchgate.net/profile/Ata_Akin/publication/281167153/figure/download/fig1/AS:391418049777669@1470332743733/Three-different-stimulus-conditions-in-the-Stroop-task-neutral-congruent-and.png?_sg=ibeklp8QZ2sbyR29ZZxbOgfS--_RjcKP_uVY36qBahzEJlnMLYPxQyzgYT2Au85eDBClhLqol0A)

个体对于不一致(incongruent)的刺激的反应正确率往往低于一致(incongruent)的刺激。

研究问题为：通过广义线性模型(Generalized linear model)检验不同刺激条件下正确率的差异。

图片来源：https://www.researchgate.net/publication/281167153_Similarity_analysis_of_functional_connectivity_with_functional_near-infrared_spectroscopy/figures?lo=1&utm_source=bing&utm_medium=organic

### (2) 数据收集

In [1]:
#加载需要使用的库
%matplotlib inline
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import pymc3 as pm

np.random.seed(123)  # 随机数种子，确保随后生成的随机数相同

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


这里我们使用的数据来自 Eisenberg et al (2019)。

为了简化问题，我们仅考虑有一个被试的数据。其中：
- worker_id 为被试编号。
- correct 为被试在 stroop 任务中每个试次判断的正确性，其中1代表判断正确，0代表判断错误。
- condition 为刺激的类别，congruent为颜色和字意一致，incongruent为颜色和字意不一致。

In [58]:
# 加载数据
data = pd.read_csv("../stroop.csv")
# 选取第一个被试在正式实验(test)中的数据
data = data[(data.worker_id == "s001") & (data.exp_stage == "test")]
# 选取数据中的判断正确率，刺激条件，和被试编号
data = data[["worker_id","correct","condition"]]
# 重置每个试次的编号
data.reset_index(inplace=True,drop=True)

In [59]:
data.head()

,worker_id,correct,condition
0,s001,1.0,congruent
1,s001,0.0,congruent
2,s001,1.0,congruent
3,s001,1.0,congruent
4,s001,1.0,congruent


对数据进行描述统计分析

可以发现，一致条件下的正确率(M = 0.875)高于不一致条件(M = 0.813)。

In [60]:
data.groupby('condition').correct.describe() 

,count,mean,std,min,25%,50%,75%,max
condition,,,,,,,,
congruent,48.0,0.8750,0.334219,0.0,1.0,1.0,1.0,1.0
incongruent,48.0,0.8125,0.394443,0.0,1.0,1.0,1.0,1.0


In [61]:
data.groupby(['condition']).correct.mean().plot.bar()
plt.show()

<Figure size 432x288 with 1 Axes>

### (3) 选择模型

在一般线性模型中，因变量被假定为服从正态分布 $y \sim Normal(\mu,sigma)$。
- 其中，y为观测项；$\mu$为预测项；sigma 为误差项。
- 预测项展开为 $\mu  = \alpha + \beta *x$，其中 x 为自变量，比如我们例子中的刺激条件。



![Image Name](https://cdn.kesci.com/upload/image/rll49b8jn9.png?imageView2/0/w/640/h/640)



在我们的例子中，由于因变量(反应的正确性)不是连续变量，因此预测变量不服从正态分布。

考虑到反应的正确性服从伯努利(Bernoulli)分布，因此我们需要广义线性模型(Generalized linear model，GLM)来扩展一般线性模型：
- 首先，GLM 可以将 $y \sim Normal(\mu,sigma)$ 扩展为 **$y \sim Bernoulli(p)$** ，使得因变量y服从伯努利分布。
- 同样，参数 p 可以与自变量联系在一起， $p  = \alpha + \beta * x$。
- 需要注意的是，由于 p 的范围被限定在0到1的，而 $\alpha + \beta * x$ 的范围为 $(-\infty, +\infty)$。我们需要通过**链接函数** 将 $\alpha + \beta * x$  映射到 p 所在的范围。
	1. 令 $\mu = \alpha + \beta *x$，$\mu$的范围为 $(-\infty, +\infty)$。
	2. $p = g(\mu)$，其中 g() 为链接函数，输出结果 p 的范围为 $(0,1)$。
	3.  最后将 p 输入到分布函数中 $y \sim Bernoulli(p)$。
![](https://docs.pymc.io/en/v4.3.0/_images/pymc-Bernoulli-1.png)


### (4)选择先验

In [62]:
# 将‘condition’进行编码，其中一致条件(congruent)编码为0，不一致条件(incongruent)编码为1。
data.condition = data.condition.map({'incongruent':1,'congruent':0})

In [63]:
# 在pymc3中，pm.Model()定义了一个新的模型对象，这个对象是模型中随机变量的容器
# 在python中，容器是一种数据结构，是用来管理特殊数据的对象
# with语句定义了一个上下文管理器，以 linear_model 作为上下文，在这个上下文中定义的变量都被添加到这个模型
with pm.Model() as GLM_model:
    # 设定先验分布: 
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1)
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data['condition'])
    # 通过链接函数对参数进行转换
    mu = alpha + beta * x                            # 对应步骤1
    p = pm.Deterministic("p", pm.math.invlogit(mu))  # 对应步骤2

    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=50)

In [64]:
az.plot_density(prior_checks['p'])
plt.show()

<Figure size 432x288 with 1 Axes>

结果发现，通过**链接函数**转换后的p值范围为 0到1。

### (5) 拟合数据

首先定义 GLM 模型：
- 其中 alpha 和 beta 为模型参数，$\alpha + \beta * x$。
- x 为自变量刺激条件(condition), 0代表一致条件，1代表不一致条件。
- 通过链接函数对参数进行转换。
    - 首先令 $\mu = \alpha + \beta * x$
    - 然后通过链接函数 pm.math.invlogit(mu)，计算出 p。
    注意，这里我们选择 logit 的反函数 invlogit作为链接函数。该链接函数使得 p 的范围为 $(0,1)$。
-  最后将 p 输入到分布函数中 $y \sim Bernoulli(p)$。

In [65]:
with pm.Model() as GLM_model:
    # 在pymc3中，pm.Model()定义了一个新的模型对象，这个对象是模型中随机变量的容器
    # 在python中，容器是一种数据结构，是用来管理特殊数据的对象
    # with语句定义了一个上下文管理器，以 linear_model 作为上下文，在这个上下文中定义的变量都被添加到这个模型
    # 定义先验
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1)
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data['condition'])
    # 线性模型：mu是确定性随机变量，这个变量的值完全由右端值确定
    mu = alpha + beta * x                            # 对应步骤1
    p = pm.Deterministic("p", pm.math.invlogit(mu))  # 对应步骤2
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", p=p, observed=data["correct"])  # 对应步骤3

In [66]:
# 展示模型结构
pm.model_to_graphviz(GLM_model)

注意：由于应用 GLM 模型时往往都会使用到链接函数，为了减轻使用者的工作量，在 pymc中可以通过设定 将 `pm.Bernoulli("y_obs", p=p)` 的设定改为 `pm.Bernoulli("y_obs", logit_p=p)` 。 完整代码如下：

In [67]:
with pm.Model() as GLM_model:
    # 定义先验
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1,shape=1)
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data['condition'])
    # 线性模型：mu是确定性随机变量，这个变量的值完全由右端值确定
    p = pm.Deterministic("p", alpha + beta * x)
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli("y_obs", logit_p=p, observed=data["correct"])

### (6)采样过程诊断

如果使用MCMC对后验进行近似，则需要首先对MCMC过程进行评估。

* 是否收敛；
* 是否接近真实的后验。

对采样过程的评估我们会采用目视检查或rhat这个指标

In [68]:
with GLM_model :
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 4 seconds.


In [69]:
az.plot_trace(trace, var_names=['alpha','beta'])
plt.show()

<Figure size 864x288 with 4 Axes>

In [70]:
az.summary(trace, var_names=['alpha','beta'], kind="diagnostics")

,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.010,0.007,1275.0,1486.0,1.0
beta[0],0.014,0.010,1152.0,1364.0,1.0


### (7)模型诊断

在MCMC有效的前提下，需要继续检验模型是否能够较好地拟合数据。

我们会使用后验预测分布通过我们得到的参数生成一批模拟数据，并将其与真实数据进行对比。

In [71]:
# 后验预测分布的计算仍在容器中进行
with GLM_model:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [73]:
# 绘制后验预测分布
az.plot_ppc(trace)
plt.show()

/opt/conda/lib/python3.7/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


<Figure size 432x288 with 1 Axes>

### (8)模型比较

当采样诊断与模型诊断说明模型是否可用后，我们可以通过模型检验来验证我们的研究问题：在不同刺激条件下(一致 vs. 不一致)个体正确率是否存在差异。

![Image Name](https://cdn.kesci.com/upload/image/rkm3pw954u.png?imageView2/0/w/960/h/960)


我们可以定义一个不考虑自变量影响的模型 `GLM_null_model`。如果之前的模型拟合优度好于该模型，那么说明自变量对模型存在影响。

In [74]:
with pm.Model() as GLM_null_model:
    # 定义先验
    p = pm.Uniform('p',0,1)  # 由于没有考虑自变量的影响，因此我们可以直接假设参数p服从0到1的均匀分布。
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Bernoulli('y_obs',p=p, observed=data['correct'])

    trace2 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [p]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 3 seconds.


In [ ]:
############################
# 练习
# 要求：完成对 GLM_null_model 的采样过程诊断与模型诊断。
############################

# 绘制各参数的采样情况
# Tips: 使用 az.plot_trace() 函数可以绘制 trace 图；使用 az.summary() 可以得到诊断统计结果


# 模型诊断
# Tips: 使用 pm.sample_posterior_predictive 可以进行后验预测检验；使用 az.plot_ppc() 可以得到后验预测检验图


当对 GLM_null_model 进行同样的检验，我们可以正式进行模型比较了。

In [75]:
# 将三个模型的采样结果进行比较
compare_dict = {"GLM_model": trace, "GLM_null_model": trace2}
# 选择loo方法进行比较
comp = az.compare(compare_dict, ic='loo')
comp

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
GLM_null_model,0,-42.581084,0.962813,0.000000,1.0,6.029162,0.000000,False,log
GLM_model,1,-43.172079,1.645279,0.590996,0.0,5.834722,0.319177,False,log


结果显示，`GLM_null_model` 模型的拟合度好于 `GLM_model`，说明不存在充分的证据表明不同刺激条件会影响个体判断的正确率。

### (9)统计推断


我们可以进一步通过统计推断印证模型比较的结果。

In [76]:
az.plot_posterior(trace, var_names=['beta'])
plt.show()

<Figure size 432x288 with 1 Axes>

参数 beta 反应了两个条件下正确率的差异。我们可以看到，该参数的后验分布的大部分包括0，说明支持两个条件下正确率存在差异的证据不足。

我们进一步查看两个参数的情况：

In [77]:
az.summary(trace, var_names=['alpha','beta'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,1.676,0.355,1.005,2.331,0.010,0.007,1275.0,1486.0,1.0
beta[0],-0.141,0.486,-1.070,0.751,0.014,0.010,1152.0,1364.0,1.0


结果发现，两个参数值的范围不太“正常”。

还记得 **链接函数**吗？
- 链接函数 g() 将 $\alpha + \beta * x$ 的范围从 $(-\infty, +\infty)$ 转换为 (0,1)
- 同时，其中的参数 $\alpha$ 和 $\beta$也被转换了，只不过他们从 (0,1) 转换为  $(-\infty, +\infty)$，所以  $\alpha$ 大于1，并且 $\beta$小于0。
- 为了他们转换回来，我们需要使用 logit 函数， $p = \frac{1}{1+e^θ}$。
具体代码如下：

In [78]:
p_congruent = 1 / (1 + np.exp(-trace.posterior["alpha"].mean())).to_pandas()
p_incongruent = 1 / (1 + np.exp(-(trace.posterior["beta"].mean()+trace.posterior["alpha"].mean()))).to_pandas()
print("alpha(一致条件) = ",p_congruent, "\n alpha+beta(不一致条件) = ", p_incongruent)

alpha(一致条件) =  0.8423416656124595 
 alpha+beta(不一致条件) =  0.8227063074419679


转换后可以发现，虽然一致条件的正确率略高于不一致条件，但这个差异并不具有统计学意义。